In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display

In [36]:
import requests

# URL du fichier Excel sur GitHub (ou autre emplacement en ligne)
url = 'https://raw.githubusercontent.com/WarTazz/club_analytics_dashboard/main/V4_mars_2024.xlsx'

# Télécharger le fichier Excel localement
response = requests.get(url)
with open('V4_mars_2024.xlsx', 'wb') as f:
    f.write(response.content)

# Charger le fichier Excel à partir du chemin local
df = pd.read_excel('V4_mars_2024.xlsx', sheet_name='BD EDF')
df.head()

ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [34]:
import locale

# Définir la locale française pour les dates
locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')

# Convertir la colonne de dates en datetime si nécessaire
df['Date'] = pd.to_datetime(df['Date'])

# Supprimer la colonne Horodateur
# df.drop("Horodateur", axis=1, inplace=True)

In [ ]:
# Extraire l'année et le numéro de semaine de la colonne de dates
df['Année'] = df['Date'].dt.year
df['Date'] = df['Date'].dt.strftime('%U')  # %U pour numéro de semaine (dimanche comme premier jour)

In [ ]:
TCD_EDF_Semaine = df.pivot_table(index=["Année", "Semaine"], values="Total EDF", aggfunc=lambda x: round(x.mean(), 1))

In [ ]:
def plot_graph(data):

    # Créer le graphique
    plt.figure(figsize=(8, 5), facecolor='lightgrey')

    # Créer une liste de chaînes représentant les semaines au format "année-semaine"
    x_values = [f"{semaine}" for annee, semaine in data.index]

    # Tracer la courbe avec les paramètres spécifiques
    plt.plot(x_values, data['Total EDF'], linestyle='-', color='r', label='Score EDF moyen', linewidth=2)



    # Ajouter un titre, des labels d'axes
    plt.title("Evolution de l'EDF moyen par semaine sur la saison 2023-2024", color='black', fontweight='bold', fontsize=12)
    plt.xlabel("Semaines", color='black', fontweight='bold')
    plt.ylabel("Score EDF moyen", fontweight='bold')

    # Définir les limites de l'axe des ordonnées (y-axis)
    plt.ylim(10, 30)

    # Ajuster les marges pour que le premier point soit aligné avec l'axe des ordonnées
    plt.margins(x=0)  # Ajustement horizontal (axe des abscisses)




    # Couleur de fond (zone de traçage)
    ax = plt.gca()
    ax.set_facecolor('beige')

    # Couleur sous la courbe
    plt.fill_between(x_values, data['Total EDF'], 0, where=(data['Total EDF'] > 0), alpha=0.3, color='blue')



    # Afficher la légende
    plt.legend(loc='upper right', fontsize='small', title='Légende', facecolor='lightgrey', frameon=True, edgecolor='black', fancybox=True)

    # Afficher le graphique
    plt.show()

In [ ]:
def update_plot(joueur):
    # Filtrer les données en fonction du joueur sélectionné
    if joueur == "Moyenne Groupe":
        filtered_data = df  # Utiliser toutes les données pour la moyenne du groupe
    else:
        filtered_data = df[df['Nom'] == joueur]  # Filtrer les données pour le joueur sélectionné

    # Agréger les données filtrées par semaine et année
    TCD_filtered = filtered_data.pivot_table(index=["Année", "Semaine"], values="Total EDF", aggfunc=lambda x: round(x.mean(), 1))

    # Afficher le graphique mis à jour
    plot_graph(TCD_filtered)

# Evolution groupe/indiv RPE et EDF

In [ ]:
# Liste des joueurs disponibles triée par ordre alphabétique
joueurs_disponibles = sorted(df['Nom'].unique().tolist())

# Insérer "Moyenne Groupe" en haut de la liste
joueurs_disponibles.insert(0, "Moyenne Groupe")

# Sélecteur de joueur
joueur_selector = widgets.Dropdown(options=joueurs_disponibles, description='Joueur :')

# Connecter le widget à la fonction de mise à jour du graphique
widgets.interact(update_plot, joueur=joueur_selector)

In [ ]:
# !voila Dashboard.ipynb